# Objectif

Dans ce notebook nous allons aborder la construction de notre API. Ce notebook sera le point d'entrée vers la conception de notre API qui sera stocké dans le dossier API.

## Import du modèle retenu

In [31]:
import joblib
import os
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

workingDir = os.getcwd()
filePath = os.path.join(workingDir,'API/model/tags_lr_compressed.joblib')
# load
mPredict = joblib.load(filePath)

filePath = os.path.join(workingDir,'API/model/tfidf.joblib')
vectorizer =  joblib.load(filePath)

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer
filePath = os.path.join(workingDir,'API/model/multilabel.joblib')
multilabel = joblib.load(filePath)


In [33]:
mPredict

OneVsRestClassifier(estimator=LogisticRegression(C=10))

## Import d'un jeu de donnée

In [34]:
import pandas as pd
import numpy as np
workingDir = os.getcwd()
dataDir = os.path.join(workingDir,'Data','posts')
filePath = os.path.join(dataDir,'QueryResults_03.csv')
df = pd.read_csv(filePath,sep=',')

In [35]:
df.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
       'ContentLicense', 'rank'],
      dtype='object')

In [36]:
dfQuestions = df[['Title','Body']]
dfTags = df.Tags

In [37]:
dfQuestions.head()

,Title,Body
0,Is it possible to use C# Object Initializers w...,<p>I'm looking at the new object initializers ...
1,Excel Conditional Formatting Self Reference,<p>I'm trying to do some conditional formattin...
2,Best way to check for current date in where cl...,<p>I'm trying to find out the most efficient (...
3,Detect an internet connection activation with ...,<p>I've been using a 3G wireless card for a wh...
4,Improve asp script performance that takes 3+ m...,<p>I use an SQL statement to remove records th...


In [38]:
dfTags.head()

0                                    <c#-3.0><factory>
1    <excel><worksheet-function><conditional-format...
2          <sql><tsql><stored-procedures><performance>
3       <delphi><winapi><wininet><internet-connection>
4                                   <sql><asp-classic>
Name: Tags, dtype: object

In [39]:
def Tag_Transform(x):
    _str = x.replace(' ', '-').replace('<','').replace('>',' ').strip(' ')
    return _str.split()

In [40]:
dfTags = dfTags.apply(Tag_Transform)

In [41]:
dfTags.head()

0                                    [c#-3.0, factory]
1    [excel, worksheet-function, conditional-format...
2          [sql, tsql, stored-procedures, performance]
3       [delphi, winapi, wininet, internet-connection]
4                                   [sql, asp-classic]
Name: Tags, dtype: object

In [74]:
def most_common(tags,listTag100):
    tags_filtered = []
    for i in range(0, len(tags)):
        if tags[i] in listTag100:
            tags_filtered.append(tags[i])
    return tags_filtered

In [76]:
dfTagsFiltered = dfTags.apply(lambda x : most_common(x,multilabel.classes_))

In [42]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models.phrases import Phrases

In [43]:
tokenizer = MWETokenizer()
tokenizer.add_mwe(('c', '#'))

In [44]:
tokenizer.tokenize(word_tokenize(dfQuestions.Title[0].lower()))

['is',
 'it',
 'possible',
 'to',
 'use',
 'c_#',
 'object',
 'initializers',
 'with',
 'factories']

In [45]:
new_stop_words=['would','want', 'please', 'help', 'can', "can't",'shall','thanks','thank','may',
                'seem','understand','error','warning','require','rather']
stop_words = stopwords.words('english')
stop_words.extend(new_stop_words)
stop_words = set(stop_words)
lemmatize=WordNetLemmatizer()
tokenizer = MWETokenizer()
tokenizer.add_mwe(('c', '#'))

file_path = os.path.join(workingDir,'API','model','bigram.pkl')
bigram_mod = Phrases.load(file_path)

def clean_text(text):
    clean_text = re.sub(r'[^A-Za-z0-9+#.\-]',' ',text.lower())
    words=word_tokenize(str(clean_text.lower()))
    words = tokenizer.tokenize(words)
    
    clean_words = [str(lemmatize.lemmatize(j)) for j in words if j not in stop_words]
    bigram_words = bigram_mod[clean_words]
    
    clean_text = ' '.join(bigram_words)
    clean_text = clean_text.replace('c_#','c#')
    
    return clean_text.strip()

In [46]:
def clean_body(body):
    _txt = BeautifulSoup(body).get_text()
    return clean_text(_txt)

In [47]:
dfFinal = dfQuestions.Title.apply(clean_text) +  dfQuestions.Body.apply(clean_body) 

In [48]:
dfFinal.head()

0    possible use c# object initializers factoryloo...
1    excel conditional formatting self referencetry...
2    best way check current date clause sql querytr...
3    detect internet connection activation delphius...
4    improve asp script performance take 3+ minute ...
dtype: object

In [49]:
X_tfidf = vectorizer.transform(dfFinal)

In [50]:
X_tfidf

<5000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 137579 stored elements in Compressed Sparse Row format>

In [51]:
yPred = mPredict.predict(X_tfidf)

In [52]:
multilabel.classes_

array(['.net', 'actionscript-3', 'ajax', 'algorithm', 'android', 'apache',
       'apache-flex', 'arrays', 'asp.net', 'asp.net-mvc', 'asp.net-mvc-2',
       'bash', 'bug', 'c', 'c#', 'c++', 'class', 'cocoa', 'cocoa-touch',
       'css', 'database', 'debugging', 'delphi', 'design-patterns',
       'discussion', 'django', 'eclipse', 'email', 'entity-framework',
       'events', 'excel', 'facebook', 'feature-request', 'file', 'flash',
       'forms', 'generics', 'git', 'google-app-engine', 'hibernate',
       'html', 'http', 'image', 'internet-explorer', 'iphone', 'java',
       'javascript', 'jquery', 'json', 'linq', 'linq-to-sql', 'linux',
       'macos', 'multithreading', 'mysql', 'nhibernate', 'objective-c',
       'oop', 'oracle', 'parsing', 'performance', 'perl', 'php', 'python',
       'qt', 'regex', 'ruby', 'ruby-on-rails', 'security', 'sharepoint',
       'silverlight', 'spring', 'sql', 'sql-server', 'sql-server-2005',
       'sql-server-2008', 'sqlite', 'string', 'support', 'svn

In [53]:
yPred[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [54]:
dfRes = pd.DataFrame(yPred)

In [55]:
dfRes.columns = multilabel.classes_

In [61]:
dfFinal = dfFinal.to_frame()

In [67]:
dfFinal = dfFinal.rename(columns={0: "Text"})

In [68]:
dfFinal['Tag_pred'] = [list(tag) for tag in  multilabel.inverse_transform(yPred)]

In [77]:
dfFinal['Tag_Filtered'] = dfTagsFiltered

In [70]:
dfFinal['Tag_orig'] = dfTags

In [78]:
dfFinal

,Text,Tag_pred,Tag_orig,Tag_Filtered
0,possible use c# object initializers factoryloo...,[c#],"[c#-3.0, factory]",[]
1,excel conditional formatting self referencetry...,[excel],"[excel, worksheet-function, conditional-format...",[excel]
2,best way check current date clause sql querytr...,[sql],"[sql, tsql, stored-procedures, performance]","[sql, tsql, performance]"
3,detect internet connection activation delphius...,[],"[delphi, winapi, wininet, internet-connection]","[delphi, winapi]"
4,improve asp script performance take 3+ minute ...,"[sql, sql-server]","[sql, asp-classic]",[sql]
...,...,...,...,...
4995,find tabselectionusing following code developi...,[jquery],"[javascript, jquery]","[javascript, jquery]"
4996,viability c# .net new standard game dev platfo...,[c#],"[c#, mono, xna]",[c#]
4997,possible animation appear wpf navigationwindow...,[],"[wpf, animation, navigationwindow]",[wpf]
4998,deploying flex project leveraging imported web...,[],"[apache-flex, flexbuilder]",[apache-flex]


Ce notebook contient les étapes majeures pour la construction de notre API. Nous avons réussi à partir d'un texte à prédire/suggérer des tags. Nous pouvons maintenant écrire notre le code source de notre API.